In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #plots
import matplotlib.pyplot as plt #more plots
import sklearn as sk #machine learning

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

> **About this kernel  **  
Before I start, I want to establish my goals for this dataset. My end goal is to predict the risk of patient developing a cardio vascular disease in the next ten years. I will start by modifying the dataset if needed to remove null data or make categorical data numerical. Next, I will make visualizations to understand the data pool I am using and the prevalence of each feature. Then, I'm going to being using models to test the importance of each feature (feature engineering). When I find the optimal set of features, I will use them in many machine learning and maybe some deep learning models and compare their effectiveness when predicting on the test data.

**Dataset Description**  
World Health Organization has estimated 12 million deaths occur worldwide, every year due to Heart diseases. Half the deaths in the United States and other developed countries are due to cardio vascular diseases. The early prognosis of cardiovascular diseases can aid in making decisions on lifestyle changes in high risk patients and in turn reduce the complications. This research intends to pinpoint the most relevant/risk factors of heart disease as well as predict the overall risk using logistic regression.


# 1. **Basic Data Analysis**

In [ ]:
#loading the data in
data = pd.read_csv("/kaggle/input/logistic-regression-heart-disease-prediction/framingham_heart_disease.csv")

Before anything is done to the data, we must take a look at the features and values included.

In [ ]:
data.head()

In [ ]:
print(data.columns)

In [ ]:
data.describe()

Here are what all the features are described as:
* male: 0 = Female; 1 = Male
* age: Age at exam time.
* education: 1 = Some High School; 2 = High School or GED; 3 = Some College or Vocational School; 4 = college
* currentSmoker: 0 = nonsmoker; 1 = smoker
* cigsPerDay: number of cigarettes smoked per day (estimated average)
* BPMeds: 0 = Not on Blood Pressure medications; 1 = Is on Blood Pressure medications
* prevalentStroke
* prevalentHyp
* diabetes: 0 = No; 1 = Yes
* totChol: mg/dL
* sysBP: mmHg
* diaBP: mmHg
* BMI: Body Mass Index calculated as: Weight (kg) / Height(meter-squared)
* heartRate: Beats/Min (Ventricular)
* glucose: mg/dL
* TenYearCHD: If the patient has had congestive heart failure in the last ten years. This will be what my models will be predicting.

There are 16 features in total. I predict that the gender, total cholesterol, TenYearCHD and age will be the most useful in predicting if a person has/will have congestive heart failure.

In [ ]:
# listing how many null values are in the dataset
data.isna().sum()

# 2. Analyzing Features using Visualizations

To begin, I plotted histograms of all the feature to visualize the distribution of data. I'm comparing features using a heatmap.

In [ ]:
# creating histograms to visualize all the data
fig = plt.figure(figsize = (40,40))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax = fig.gca()
data.hist(ax = ax)

In [ ]:
# creating a heatmap to find the correlation between each feature
sns.set(font_scale=3)
def plot_corr( df ):
    corr = df.corr()
    _, ax=plt.subplots( figsize=(50,25) )
    cmap = sns.diverging_palette( 240 , 10 , as_cmap = True)
    _ = sns.heatmap(corr,cmap=cmap,square=True, cbar_kws = {'shrink': .9}, fmt= '.1f', ax=ax, annot=True)
    
plot_corr(data)


# 3. Cleaning the dataset

We have to find a way to effectively remove or replace the null values so that models can be applied.

The education column could be very subjective, has many null values and has no correlation to the other feature, so I'm going to drop it.

In [ ]:
data = data.drop(['education'], axis=1)

In [ ]:
data.head()

I'm not sure what to do with the glucose column since so many values are missing, but glucose is very important when predicting heart disease so I'll impute the values with a SimpleImputer. Imputation is when you replace the null values with the mean or any other value. Total cholesterol, BMI and cigs per day is useful, so we'll impute them too.

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
my_imputer.fit(data)
imputed_data = pd.DataFrame(my_imputer.transform(data))

# Imputation removed column names; put them back
imputed_data.columns = data.columns

In [ ]:
imputed_data.isna().sum()

### Splitting the training and testing data

In order to actually test the accuracy of the models, we must set some data aside for testing and some data to train the models.

In [ ]:
from sklearn.model_selection import train_test_split

y = imputed_data['TenYearCHD']
X = imputed_data.drop(['TenYearCHD'], axis = 1)

# divide
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=14)

# 4. Standardization

In [ ]:
#Creating a set of features that are standardized
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = pd.DataFrame(scaler.fit_transform(X_train))
X_test_std = pd.DataFrame(scaler.transform(X_test))

X_train_std.columns = X_train.columns
X_test_std.columns = X_test.columns

X_train_std

# 5. Normalization

A problem with this dataset that's also mentioned in another kernel (https://www.kaggle.com/lauriandwu/machine-learning-heart-disease-framingham/comments) is that there is no balance between people with and without heart disease, so models will be more likely to predict that a patient doesn't have heart disease without a thourough analysis with this data. I'm going to try to deal with this unbalance by splitting the training and testing data and then balancing the training data. I will undersample to balance the training data.

In [ ]:
# Shuffle df
shuffled_df = imputed_data.sample(frac=1,random_state=4)

# Put all the fraud class in a separate dataset.
CHD_df = shuffled_df.loc[shuffled_df['TenYearCHD'] == 1]

#Randomly select 492 observations from the non-fraud (majority class)
non_CHD_df = shuffled_df.loc[shuffled_df['TenYearCHD'] == 0].sample(n=611,random_state=42)

# Concatenate(join) both dataframes again
normalized_df = pd.concat([CHD_df, non_CHD_df])

# plot new count
sns.countplot(normalized_df.TenYearCHD, palette="OrRd")
plt.box(False)
plt.xlabel('Heart Disease No/Yes',fontsize=11)
plt.ylabel('Patient Count',fontsize=11)
plt.title('Count Outcome Heart Disease after Resampling\n')
#plt.savefig('Balance Heart Disease.png')
plt.show()

When you get to the model training section, you may notice that I don't use these normalized values for training. I tried using the normalized values and the 4 types of accuracies were almost around the same percentages, which is good, however all the accuracies were between 50%~70%. So, I decided to not use the normalized data.

# 6. Feature selection

The features in this dataset seem all important, however reducing features to the top 5 or 10 can increase the accuracy score. I'm going to score models with limited features and full feature.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# find best scored 6 features
select_feature = SelectKBest(chi2, k=10).fit(X_train, y_train)

dfscores = pd.DataFrame(select_feature.scores_)
dfcolumns = pd.DataFrame(X_train.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
top_featureScores = featureScores.nlargest(10,'Score')
print(top_featureScores)

In [ ]:
# visualizing feature selection
plt.figure(figsize=(20,5))
sns.barplot(x='Specs', y='Score', data=featureScores, palette = "GnBu_d")
plt.title('Feature importance', fontsize=16)
plt.xlabel('\n Features', fontsize=14)
plt.ylabel('Importance \n', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

I'm going to compare using 6 features and all features in each model.

In [ ]:
#Making a dataset with only the top 10 features
best_feature_names = top_featureScores['Specs']

X_best_train = X_train[best_feature_names]
X_best_test = X_test[best_feature_names]
y_best_train = y_train[best_feature_names]
y_best_test = y_test[best_feature_names]

X_best_train

# 7. Applying models
***
The models used:
* Linear regression
* Random forest
* KNN
* XGBoost  

I will be using a grid search to tune the parameters of each model.

## 1. **Logistic Regression**

### Tuning the parameters of the model

In [ ]:
from sklearn.linear_model import LogisticRegression

LR_model= LogisticRegression()

tuned_parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] ,
              'penalty':['l1','l2']
                   }

In [ ]:
# Using a gridsearch to tune parameters
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(LR_model, tuned_parameters,cv=10)

GS.fit(X_train_std, y_train)

In [ ]:
print(GS.best_params_)

In [ ]:
# Making a new model with the best parameters
LR_model= LogisticRegression(C=0.1, penalty='l2')

### Standardized predictions

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

#Fitting the model to the standardized data
LR_model.fit(X_train_std, y_train)

#Using the model to predict
y_pred = LR_model.predict(X_test_std)
y_pred

#Getting the accuracies

# check accuracy: Accuracy: Overall, how often is the classifier correct? Accuracy = (True Pos + True Negative)/total
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for LogReg is: {round(acc,3)*100}%")

# f1 score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for LogReg is: {round(f1,3)*100}%")

# Precision score: When it predicts yes, how often is it correct? Precision=True Positive/predicted yes
precision = precision_score(y_test, y_pred)
print(f"The precision score for LogReg is: {round(precision,3)*100}%")

# recall score: True Positive Rate(Sensitivity or Recall): When it’s actually yes, how often does it predict yes? True Positive Rate = True Positive/actual yes
recall = recall_score(y_test, y_pred)
print(f"The recall score for LogReg is: {round(recall,3)*100}%")

### Non-standardized predictions

In [ ]:
from sklearn.metrics import accuracy_score

#Fitting the model to the non standardized data
LR_model.fit(X_train, y_train)

#Using the model to predict
y_pred = LR_model.predict(X_test)

# check accuracy: Accuracy: Overall, how often is the classifier correct? Accuracy = (True Pos + True Negative)/total
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for LogReg is: {round(acc,3)*100}%")

# f1 score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for LogReg is: {round(f1,3)*100}%")

# Precision score: When it predicts yes, how often is it correct? Precision=True Positive/predicted yes
precision = precision_score(y_test, y_pred)
print(f"The precision score for LogReg is: {round(precision,3)*100}%")

# recall score: True Positive Rate(Sensitivity or Recall): When it’s actually yes, how often does it predict yes? True Positive Rate = True Positive/actual yes
recall = recall_score(y_test, y_pred)
print(f"The recall score for LogReg is: {round(recall,3)*100}%")

### Limited Feature Model

In [ ]:
from sklearn.metrics import accuracy_score

#Fitting the model to the standardized data
LR_model.fit(X_best_train, y_train)

#Using the model to predict
y_pred = LR_model.predict(X_best_test)
y_pred

# check accuracy: Accuracy: Overall, how often is the classifier correct? Accuracy = (True Pos + True Negative)/total
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for LogReg is: {round(acc,3)*100}%")

# f1 score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for LogReg is: {round(f1,3)*100}%")

# Precision score: When it predicts yes, how often is it correct? Precision=True Positive/predicted yes
precision = precision_score(y_test, y_pred)
print(f"The precision score for LogReg is: {round(precision,3)*100}%")

# recall score: True Positive Rate(Sensitivity or Recall): When it’s actually yes, how often does it predict yes? True Positive Rate = True Positive/actual yes
recall = recall_score(y_test, y_pred)
print(f"The recall score for LogReg is: {round(recall,3)*100}%")

The standardized data has a slightly higher accuracy and the limited feature model has a lower accuracy than the standardized model. Overall, the full, standardized model predicts the best with linear regression.

## 2. Random Forest

### Tuning Parameters

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#initiating a new model
rf = RandomForestClassifier(random_state=1)

#making a list of parameters for the grid search to compare
tuned_parameters = {'n_estimators': [100, 500, 1000]}

#Making a grid search model
GS_rf=GridSearchCV(rf, tuned_parameters, cv=10)

#Fitting the grid search
# GS_rf.fit(X_train_std, y_train)

#Printing the best features
# print(GS_rf.best_params_)

# I commented the last lines of code out so that I wouldn't have to run it again since I already know the best paramters

In [ ]:
# adding the best parameters
rf = RandomForestClassifier(random_state=1, n_estimators = 1000)

### Standardized

In [ ]:
#fitting the model
rf.fit(X_train_std, y_train)

#making predictions
y_pred = rf.predict(X_test_std)

# check accuracy: Accuracy: Overall, how often is the classifier correct? Accuracy = (True Pos + True Negative)/total
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for RandomForest is: {round(acc,3)*100}%")

# f1 score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for RandomForest is: {round(f1,3)*100}%")

# Precision score: When it predicts yes, how often is it correct? Precision=True Positive/predicted yes
precision = precision_score(y_test, y_pred)
print(f"The precision score for RandomForest is: {round(precision,3)*100}%")

# recall score: True Positive Rate(Sensitivity or Recall): When it’s actually yes, how often does it predict yes? True Positive Rate = True Positive/actual yes
recall = recall_score(y_test, y_pred)
print(f"The recall score for RandomForest is: {round(recall,3)*100}%")

### Non-standardized

In [ ]:
#fitting the model
rf.fit(X_train, y_train)

#making predictions
y_pred = rf.predict(X_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for RandomForest is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for RandomForest is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for RandomForest is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for RandomForest is: {round(recall,3)*100}%")

### Limited

In [ ]:
#fitting the model
rf.fit(X_best_train, y_train)

#making predictions
y_pred = rf.predict(X_best_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for RandomForest is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for RandomForest is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for RandomForest is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for RandomForest is: {round(recall,3)*100}%")

## 3. KNN

### Tuning

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#Initializing the model
knn = KNeighborsClassifier()

#The parameters
my_params={'n_neighbors': [1,2,5,10,15, 20, 21, 25, 30]}

#creating the gridsearch
GS_knn = GridSearchCV(knn, my_params, cv=10)

#fitting the model
GS_knn.fit(X_train_std, y_train)

#finding the best parameters
print(GS_knn.best_params_)

In [ ]:
#making a tuned model
knn = KNeighborsClassifier(n_neighbors=20)

### Standardized

In [ ]:
#fitting the model
knn.fit(X_train_std, y_train)

#making predictions
y_pred = knn.predict(X_test_std)

# check accuracy: Accuracy: Overall, how often is the classifier correct? Accuracy = (True Pos + True Negative)/total
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for KNN is: {round(acc,3)*100}%")

# f1 score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for KNN is: {round(f1,3)*100}%")

# Precision score: When it predicts yes, how often is it correct? Precision=True Positive/predicted yes
precision = precision_score(y_test, y_pred)
print(f"The precision score for KNN is: {round(precision,3)*100}%")

# recall score: True Positive Rate(Sensitivity or Recall): When it’s actually yes, how often does it predict yes? True Positive Rate = True Positive/actual yes
recall = recall_score(y_test, y_pred)
print(f"The recall score for KNN is: {round(recall,3)*100}%")

### Non-Standardized

In [ ]:
#fitting the model
knn.fit(X_train, y_train)

#making predictions
y_pred = knn.predict(X_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for KNN is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for KNN is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for KNN is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for KNN is: {round(recall,3)*100}%")

### Limited

In [ ]:
#fitting the model
knn.fit(X_best_train, y_train)

#making predictions
y_pred = knn.predict(X_best_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for KNN is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for KNN is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for KNN is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for KNN is: {round(recall,3)*100}%")

For the KNN model, the limited feature model has the highest accuracy and is overall the best model.

## 4. XGBoost

### **Tuning parameters**

I'm not using gridsearch to tune the XGBoost model since it kind of self-tunes. I'm going to change some parameters and see if the accuracy increases. Here's more info on the XGBoost model: https://www.kaggle.com/dansbecker/xgboost.

### Standardized

In [ ]:
from xgboost import XGBRegressor

#creating a new model with experimental parameters
XGB = XGBRegressor(n_estimators=1000, early_stopping_rounds=5, learning_state=0.01, objective='binary:hinge')

#fitting the model
XGB.fit(X_train_std, y_train)

#making predcitions
y_pred = XGB.predict(X_test_std)

# check accuracy: Accuracy: Overall, how often is the classifier correct? Accuracy = (True Pos + True Negative)/total
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for XGB is: {round(acc,3)*100}%")

# f1 score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for XGB is: {round(f1,3)*100}%")

# Precision score: When it predicts yes, how often is it correct? Precision=True Positive/predicted yes
precision = precision_score(y_test, y_pred)
print(f"The precision score for XGB is: {round(precision,3)*100}%")

# recall score: True Positive Rate(Sensitivity or Recall): When it’s actually yes, how often does it predict yes? True Positive Rate = True Positive/actual yes
recall = recall_score(y_test, y_pred)
print(f"The recall score for XGB is: {round(recall,3)*100}%")

### Non-Standardized

In [ ]:
#creating a new model with experimental parameters
XGB = XGBRegressor(n_estimators=1000, early_stopping_rounds=5, learning_state=0.05, objective='binary:hinge')

#fitting the model
XGB.fit(X_train, y_train)

#making predcitions
y_pred = XGB.predict(X_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for XGB is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for XGB is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for XGB is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for XGB is: {round(recall,3)*100}%")

### Limited

In [ ]:
#creating a new model with experimental parameters
XGB = XGBRegressor(n_estimators=1000, early_stopping_rounds=5, learning_state=0.05, objective='binary:hinge')

#fitting the model
XGB.fit(X_best_train, y_train)

#making predcitions
y_pred = XGB.predict(X_best_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for XGB is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for XGB is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for XGB is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for XGB is: {round(recall,3)*100}%")

In conclusion, the XGBoost does not perform better than the other models. Out of the three types of data, the limited one has the highest accuracy.

## 5. Support Vector Machine (SVM)

I decided to test if the normalized data would improve all other scores other than accuracy for the SVM. It did, but the accuracy score went form 84% to 65.3%, so I decided not to use the normalized data.

In [ ]:
'''
y = normalized_df['TenYearCHD']
X = normalized_df.drop(['TenYearCHD'], axis = 1)

# divide
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=14)

#Creating a set of features that are standardized
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = pd.DataFrame(scaler.fit_transform(X_train))
X_test_std = pd.DataFrame(scaler.transform(X_test))

X_train_std.columns = X_train.columns
X_test_std.columns = X_test.columns

X_train_std
'''

### Standardized

In [ ]:
from sklearn.svm import SVC

#initialize model
svm = SVC()

#fit model
svm.fit(X_train_std, y_train)

#make predictions
y_pred = svm.predict(X_test_std)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for SVM is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for SVM is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for SVM is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for SVM is: {round(recall,3)*100}%")

### Non-Standardized

In [ ]:
#initialize model
svm = SVC()

#fit model
svm.fit(X_train, y_train)

#make predictions
y_pred = svm.predict(X_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for SVM is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for SVM is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for SVM is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for SVM is: {round(recall,3)*100}%")

### Limited

In [ ]:
#initialize model
svm = SVC()

#fit model
svm.fit(X_best_train, y_train)

#make predictions
y_pred = svm.predict(X_best_test)

# check accuracy
acc = accuracy_score(y_test, y_pred)
print(f"The accuracy score for SVM is: {round(acc,3)*100}%")

# f1 score
f1 = f1_score(y_test, y_pred)
print(f"The f1 score for SVM is: {round(f1,3)*100}%")

# Precision score
precision = precision_score(y_test, y_pred)
print(f"The precision score for SVM is: {round(precision,3)*100}%")

# recall score
recall = recall_score(y_test, y_pred)
print(f"The recall score for SVM is: {round(recall,3)*100}%")

# Conclusion

I'm going to compare the accuracies from the best of each model using a bar plot.

In [ ]:
import matplotlib.font_manager
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

# Making a dataframe of the accuracies
a = {'Random Forest Classifier': [84.89], 'K-Nearest Neighbours': [84.89], 'Logistic Regression': [85.5], 'XGBoost':[83.3], 'SVM':[84.7]}
accuracies = pd.DataFrame(data=a)

# making bar plot comparing the accuracies of the models
sns.set(font_scale=1)
ax = accuracies.plot.bar(
    figsize= (10, 5),
    fontsize=14)
plt.xticks(rotation=0, fontsize=14)
plt.xlabel('Models', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
x_labels = ['']
xticks = [-0.20, -0.1, 0.02, 0.14, ]
ax.set_xticks(xticks)
ax.set_xticklabels(x_labels, rotation=0)
axbox = ax.get_position()
plt.legend(loc = (axbox.x0 + 0.65, axbox.y0 + 0.70), fontsize=14)
plt.title('Accuracies of Each Model Predicting CHD')
ax.set_facecolor('xkcd:white')
ax.set_facecolor(('#ffffff'))
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')

# 8. Will you develop CHD?

Enter you information as the variables to see if you'll get CHD. I've inputted my values (and average values for values I could not get at the moment)

In [ ]:
my_predictors = []
parameters=['sysBP', 'glucose','age','totChol','cigsPerDay','diaBP','prevalentHyp','diabetes','BPMeds','male']

age = 17 ##"Patient's age: >>>
my_predictors.append(age)
male = 0#"Patient's gender. male=1, female=0:
my_predictors.append(male)
cigsPerDay =0 ##"Patient's smoked cigarettes per day:
my_predictors.append(cigsPerDay)
sysBP = 100##"Patient's systolic blood pressure:
my_predictors.append(sysBP)
diaBP = 80##"Patient's diastolic blood pressure
my_predictors.append(diaBP)
totChol = 130##"Patient's cholesterin level:
my_predictors.append(totChol)
prevalentHyp =0 ##"Was Patient hypertensive? Yes=1, No= 0
my_predictors.append(prevalentHyp)
diabetes = 0##"Did Patient have diabetes? Yes=1, No=0
my_predictors.append(diabetes)
glucose = 100##"What is the Patient's glucose level?
my_predictors.append(diabetes)
BPMeds = 0##"Has Patient been on Blood Pressure Medication? Yes=1, No=0
my_predictors.append(BPMeds)

#adding the data to the dataset
my_data = dict(zip(parameters, my_predictors))
my_df = pd.DataFrame(my_data, index=[0])

my_y_pred = knn.predict(my_df)
print('Result:')
if my_y_pred == 1:
    print("The patient will develop a heart disease.")
if my_y_pred == 0:
    print("The patient will not develop a heart disease.")

Yay! I won't get heart disease.